In [2]:
from langchain_chroma import Chroma
import fitz
import torch
from langchain_community.document_loaders.parsers import PyMuPDFParser
from langchain_core.documents.base import Blob
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import CLIPProcessor,CLIPModel
from PIL import Image
import base64
import io

In [53]:
file_path="../data/attention-is-all-you-need-Paper.pdf"

model=CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor=CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30
)
docs=fitz.open(file_path)

config.json: 0.00B [00:00, ?B/s]

c:\Users\SWEETY\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SWEETY\.cache\huggingface\hub\models--openai--clip-vit-base-patch32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  )


pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [55]:
def embed_text(text:str):
    inputs=processor(
        text=text,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=80
    )
    with torch.no_grad():
        features=model.get_text_features(**inputs)

        #normalize embeddings
        features=features/features.norm(dim=-1,keepdim=True)
        return features.squeeze().numpy()

In [ ]:
def embed_image(image_data):
    if isinstance(image_data,str):
        image=Image.Open(image_data).convert("RGB")
    else: 
        image=image_data

    inputs=processor(images=image,return_tensors='pt')

    with torch.no_grad():
        features=model.get_image_features(**inputs)

        features=features/features.norm(dim=-1,keepdim=True)

        return features.squeeze().cpu().numpy()

In [45]:
docs.page_count
all_docs=[]
all_embeddings=[]
image_data_store={}

In [ ]:
for i,page in enumerate(docs):
    
    ### processing text cotent of the page
    text=page.get_text()
    if text.strip():
        temp_doc=Document(page_content=text,metadata={"page":i,"type":"text"})
        text_chunks=splitter.split_documents([temp_doc])

        for chunk in text_chunks:
            all_docs.append(chunk)
            embedding=embed_text(chunk)
            all_embeddings.append(embedding)
    

    #### processing image content of the page

    for img_i,img in enumerate(page.get_images(full=True)):
        try:
            xref=img[0]
            base_image=docs.extract_image(xref)
            image_bytes=base_image.get('image')

            ###convert to PIL Image
            pil_image=Image.open(io.BytesIO(image_bytes)).convert('RGB')

            image_id=f"page: {i}_img_{img_i}"

            #save the image bytes in base64 format for mdoel use
            buffered=io.BytesIO()
            pil_image.save(buffered,format="PNG")
            img_base64=base64.b64encode(buffered.getValue()).decode()
            image_data_store[image_id]=img_base64

            #now create embeddings of image and store

            all_embeddings.append(embed_image(pil_image))
            img_doc=Document(
                page_content=f"[Image: {image_id}]",
                metadata={"page":i,"type":"image","image_id":image_id}
            )
            all_docs.append(img_doc)
        except Exception as e:
            print(f"Error processing image {img_i} on page {i}: {e}")
            continue   
            

docs.close()        

[]
[]
[(94, 95, 1520, 2239, 8, 'DeviceRGB', '', 'Im1', 'FlateDecode', 0)]
[(102, 103, 835, 1282, 8, 'DeviceRGB', '', 'Im3', 'FlateDecode', 0), (104, 105, 445, 884, 8, 'DeviceRGB', '', 'Im2', 'FlateDecode', 0)]
[]
[]
[]
[]
[]
[]
[]
